# Downloading the dataset

We are using the Face Forensics++ (FF++) dataset

In [ ]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("sanikatiwarekar/deep-fake-detection-dfd-entire-original-dataset")

# print("Path to dataset files:", path)

/Users/jordan/Desktop/Live_Projects/History_of_Man/proactives_venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 22.5G/22.5G [23:54<00:00, 16.8MB/s]  


Extracting files...
Path to dataset files: /Users/jordan/.cache/kagglehub/datasets/sanikatiwarekar/deep-fake-detection-dfd-entire-original-dataset/versions/1


No CSV file found in dataset directory: /Users/jordan/.cache/kagglehub/datasets/sanikatiwarekar/deep-fake-detection-dfd-entire-original-dataset/versions/1


In [ ]:
import os
from dotenv import load_dotenv
#store the paths to the real and fake images in an env file

load_dotenv()

real_path = os.getenv('REAL_PATH')
fake_path = os.getenv('DEEPFAKE_PATH')

def count_mp4_files(folder):
    mp4_count = 0
    for root, dirs, files in os.walk(folder):
        mp4_count += sum(1 for f in files if f.endswith('.mp4'))
    return mp4_count

real_mp4_count = count_mp4_files(real_path)
fake_mp4_count = count_mp4_files(fake_path)

print(f'Real folder mp4 count: {real_mp4_count}')
print(f'Fake folder mp4 count: {fake_mp4_count}')

Real folder mp4 count: 363
Fake folder mp4 count: 3068


Use the EfficientNet model for feature extraction

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
import numpy as np

def get_efficientnet_feature_extractor():
    """
    Returns an EfficientNetB0 model that outputs feature vectors instead of class predictions.
    """
    # Load EfficientNetB0 without the top classification layer
    base_model = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')
    # The output will be the global average pooled features
    feature_extractor = Model(inputs=base_model.input, outputs=base_model.output)
    return feature_extractor

def extract_face_features(face_images, feature_extractor):
    """
    Given a batch of face images, returns their EfficientNet feature vectors.
    face_images: numpy array of shape (batch_size, height, width, channels)
    feature_extractor: model returned by get_efficientnet_feature_extractor()
    """
    # EfficientNet expects images scaled to [0, 255] and size 224x224
    # You may need to preprocess your images accordingly
    features = feature_extractor.predict(face_images)
    return features